In [32]:
import numpy as np
import tensorflow as tf
#import pydot
from IPython.display import SVG
import matplotlib.pyplot as plt
import scipy.misc
import datetime

from utils import load_dataset, convert_to_one_hot
%matplotlib inline

In [2]:
K = tf.keras.backend
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [3]:
# Residual newtwork solves the problem of vanishing gradient and gradient exploding
#In ResNets, a "shortcut" or a "skip connection" allows the gradient to be directly backpropagated to earlier layers:  


# 1) The Identity Block


The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say $a^{[l]}$) has the same dimension as the output activation (say $a^{[l+2]}$). To flesh out the different steps of what happens in a ResNet's identity block, here is an alternative diagram showing the individual steps

In [11]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # retrive filters 
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # first component of the main path
    X = tf.keras.layers.Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1),
                               padding = 'valid', name = conv_name_base + '2a',
                               kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(X)
    
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    # Second Component of the main path
    X = tf.keras.layers.Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1),
                               padding = 'same', name = conv_name_base + '2b',
                               kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(X)
    
    X = tf.keras.layers.BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    # third component of the main path
    X = tf.keras.layers.Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1),
                               padding = 'valid', name = conv_name_base + '2c',
                               kernel_initializer = tf.keras.initializers.GlorotUniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    
    # Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    return X



# 2) Convolution Block
You can use this type of block when the input and output dimensions don't match up

In [12]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    
    X = tf.keras.layers.Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    
    X = tf.keras.layers.Conv2D(F2, (f, f), strides = (1,1), padding='same', name = conv_name_base + '2b', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = tf.keras.layers.Activation('relu')(X)

    X = tf.keras.layers.Conv2D(F3, (1, 1), strides = (1,1), padding='valid', name = conv_name_base + '2c', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = tf.keras.layers.Conv2D(F3, (1, 1), strides = (s,s), padding='valid', name = conv_name_base + '1', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = tf.keras.layers.BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Add shortcut value to main path
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)
    
    
    return X

# Building 50 layers ResNet

In [13]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = tf.keras.layers.Input(input_shape)

    
    # Zero-Padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = tf.keras.layers.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    X = tf.keras.layers.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = tf.keras.layers.AveragePooling2D()(X)

    # output layer
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = tf.keras.initializers.glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = tf.keras.models.Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [14]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)

In [33]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


# Testing Own Image

In [23]:
img_path = 'images/5.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
x = tf.keras.preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.imagenet_utils.preprocess_input(x)
print('Input image shape:', x.shape)
print("class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = ")
print(model.predict(x))

Input image shape: (1, 64, 64, 3)
class prediction vector [p(0), p(1), p(2), p(3), p(4), p(5)] = 
[[1. 0. 0. 0. 0. 0.]]


# Tensorboard

In [28]:
# Load the tensorboard notebook extension
%load_ext tensorboard

In [29]:
# clear any logs from previous run
!rm -rf ./logs/

In [30]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(x = X_train, y = Y_train,
          epochs = 10,
          validation_data = (X_test, Y_test),
          callbacks = [tensorboard_callbacks],
          batch_size = 32)

Train on 1080 samples, validate on 120 samples
Epoch 1/10
1080/1080 [==============================] - 75s 69ms/sample - loss: 0.6873 - accuracy: 0.8231 - val_loss: 3618.3951 - val_accuracy: 0.1667
Epoch 2/10
1080/1080 [==============================] - 59s 55ms/sample - loss: 0.3302 - accuracy: 0.8898 - val_loss: 138.5140 - val_accuracy: 0.1667
Epoch 3/10
1080/1080 [==============================] - 59s 55ms/sample - loss: 0.2851 - accuracy: 0.9028 - val_loss: 126.5282 - val_accuracy: 0.1750
Epoch 4/10
1080/1080 [==============================] - 57s 53ms/sample - loss: 0.3743 - accuracy: 0.9259 - val_loss: 18915.6203 - val_accuracy: 0.1667
Epoch 5/10
1080/1080 [==============================] - 58s 53ms/sample - loss: 0.3134 - accuracy: 0.9213 - val_loss: 82500.7766 - val_accuracy: 0.1667
Epoch 6/10
1080/1080 [==============================] - 57s 52ms/sample - loss: 0.5049 - accuracy: 0.8630 - val_loss: 1296747.2417 - val_accuracy: 0.1667
Epoch 7/10
1080/1080 [======================

In [38]:
model.save('handsign.h5')

In [39]:
model = tf.keras.models.load_model('handsign.h5')

In [40]:
%tensorboard --logdir logs/fit